In [1]:
# importing libraries to work with the data
import pandas as pd
import numpy as np
import datetime
import requests
import json

# settings to view all rows and columns
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 999)

In [2]:
# getting the list of subject IDs having adverse effect
subjects_ae = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/list')
subjects_ae_json = subjects_ae.json()
subjects_ae = pd.DataFrame(subjects_ae_json['data'], columns = ['subject_id'])

In [3]:
subjects_ae.head()

,subject_id
0,76809
1,105792
2,117691
3,120205
4,130301
5,132645
6,138700
7,156724
8,166621
9,168918


In [4]:
# getting AE data for one subject ID (2660588)
response_ae = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/2660588/list')
response_ae_json = response_ae.json()
ae_df = pd.DataFrame(response_ae_json['data'])

In [5]:
# converting the date column into a readable format to clean
ae_df['aestdat'] = pd.to_datetime(ae_df['aestdat'], errors = 'coerce', format = '%d-%b-%y')
ae_df['aeendat'] = pd.to_datetime(ae_df['aeendat'], errors = 'coerce', format = '%d-%b-%y')
ae_df.head()

,studyid,siteid,subjid,subjectid,visitindex,visitnam,formname,formid,formidx,formindex,dov,aemeyn,aemeyn_c,aecat,aecat_c,aeterm,aemedrg,aeinpkg,aestdat,aestdat_dts,aestdat_nd,aestdme,aestdme_c,aeendat,aeendat_dts,aeendat_nd,aedis,aedis_c,aemerae,aemerae_c,aespid,aendgiv,aendgiv_c,aecmgiv,aecmgiv_c,aecmgiv_nd,aerefid,compterm,aellt,aedecod,aehlt,aehlgt,aesoc,aemedrg_c,aespid_1,aeyn,aeyn_c,aeongo_c,aeongo,aetoxgr_c,aetoxgr,aeser_c,aeser,aerme_c,aerme,aetrt1,aetrt3_c,aerel1_c,aerel1,aerel1_nd,aeacn1_c,aeacn1,aeacn1_1_c,aeacn1_1,aetrt2,aetrt2_nd,aetrt2_c,aerel2_c,aerel2,aerel2_nd,aeacn2_c,aeacn2,aeacn2_1_c,aeacn2_1,aetrt3,aetrt1_c,aerel3_c,aerel3,aeacn3_c,aeacn3,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout
0,StudyHack,5086,50865017,2660588,1,AECM,AE202,38768,26,6.150000e+14,,,,ADVERSE EVENT,1,INVESTIGATIONS PLATELET COUNT INCREASED,,None,2019-04-16,4/16/2019,,,None,2019-04-26,4/26/2019,,NO,N,,,26,NO,N,NO,N,,NaN,INVESTIGATIONS PLATELET COUNT INCREASED,Platelet count increased,Platelet count increased,Platelet analyses,Haematology investigations (incl blood groups),Investigations,None,None,YES,Y,N,NO,2.0,1.0,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug1,1,1,NOT RELATED,NaN,,2.0,DOSE NOT CHANGED,3.0,OTHER,,,3.0,RECOVERED/RESOLVED
1,StudyHack,5086,50865017,2660588,1,AECM,AE202,38768,27,6.150000e+14,,,,ADVERSE EVENT,1,NEUTROPHIL COUNT DECREASE,,None,2019-04-16,4/16/2019,,,None,2019-04-18,4/18/2019,,NO,N,,,27,NO,N,YES,Y,,NaN,NEUTROPHIL COUNT DECREASE,Neutrophil count decreased,Neutrophil count decreased,White blood cell analyses,Haematology investigations (incl blood groups),Investigations,None,None,YES,Y,N,NO,4.0,3.0,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3.0,RECOVERED/RESOLVED
2,StudyHack,5086,50865017,2660588,1,AECM,AE202,38768,28,6.150000e+14,,,,ADVERSE EVENT,1,ANEMIA,,None,2019-04-18,4/18/2019,,,None,2019-09-18,9/18/2019,,NO,N,,,28,NO,N,NO,N,,NaN,ANEMIA,Anemia,Anaemia,Anaemias NEC,Anaemias nonhaemolytic and marrow depression,Blood and lymphatic system disorders,None,None,YES,Y,N,NO,3.0,2.0,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3.0,RECOVERED/RESOLVED
3,StudyHack,5086,50865017,2660588,1,AECM,AE202,38768,1,5.780000e+14,,,,ADVERSE EVENT,1,DIARRHEA,,None,2019-02-13,2/13/2019,,,None,2019-02-14,2/14/2019,,NO,N,,,1,NO,N,NO,N,,NaN,DIARRHEA,Diarrhea,Diarrhoea,Diarrhoea (excl infective),Gastrointestinal motility and defaecation cond...,Gastrointestinal disorders,None,None,YES,Y,N,NO,3.0,2.0,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3.0,RECOVERED/RESOLVED
4,StudyHack,5086,50865017,2660588,1,AECM,AE202,38768,2,5.780000e+14,,,,ADVERSE EVENT,1,DIARRHEA,,None,2019-02-14,2/14/2019,,,None,NaT,,,NO,N,,,2,NO,N,NO,N,,NaN,DIARRHEA,Diarrhea,Diarrhoea,Diarrhoea (excl infective),Gastrointestinal motility and defaecation cond...,Gastrointestinal disorders,None,None,YES,Y,Y,YES,2.0,1.0,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,2.0,NOT RECOVERED/NOT RESOLVED
5,StudyHack,5086,50865017,2660588,1,AECM,AE202,38768,3,5.780000e+14,,,,ADVERSE EVENT,1,FEBRILE NEUTROPENIA,,None,2019-02-16,2/16/2019,,,None,2019-02-27,2/27/2019,,NO,N,,,3,NO,N,YES,Y,,NaN,FEBRILE NEUTROPENIA,Febrile neutropenia,Febrile neutropenia,Neutropenias,White blood cell disorders,Blood and lymphatic system disorders,None,None,YES,Y,N,NO,4.0,3.0,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,

In [6]:
# getting AE data for second subject ID (130301)
response_ae2 = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/130301/list')
response_ae_json2 = response_ae2.json()
ae_df2 = pd.DataFrame(response_ae_json2['data'])

In [7]:
# converting the date column into a readable format to clean
ae_df2['aestdat'] = pd.to_datetime(ae_df2['aestdat'], errors = 'coerce', format = '%d-%b-%y')
ae_df2['aeendat'] = pd.to_datetime(ae_df2['aeendat'], errors = 'coerce', format = '%d-%b-%y')
ae_df2.head()

,studyid,siteid,subjid,subjectid,visitindex,visitnam,formname,formid,formidx,formindex,dov,aemeyn,aemeyn_c,aecat,aecat_c,aeterm,aemedrg,aeinpkg,aestdat,aestdat_dts,aestdat_nd,aestdme,aestdme_c,aeendat,aeendat_dts,aeendat_nd,aedis,aedis_c,aemerae,aemerae_c,aespid,aendgiv,aendgiv_c,aecmgiv,aecmgiv_c,aecmgiv_nd,aerefid,compterm,aellt,aedecod,aehlt,aehlgt,aesoc,aemedrg_c,aespid_1,aeyn,aeyn_c,aeongo_c,aeongo,aetoxgr_c,aetoxgr,aeser_c,aeser,aerme_c,aerme,aetrt1,aetrt3_c,aerel1_c,aerel1,aerel1_nd,aeacn1_c,aeacn1,aeacn1_1_c,aeacn1_1,aetrt2,aetrt2_nd,aetrt2_c,aerel2_c,aerel2,aerel2_nd,aeacn2_c,aeacn2,aeacn2_1_c,aeacn2_1,aetrt3,aetrt1_c,aerel3_c,aerel3,aeacn3_c,aeacn3,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout
0,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,1,5.550000e+14,,,,ADVERSE EVENT,1,ANOREXIA,,None,2018-06-08,6/8/2018,,,None,2018-06-16,6/16/2018,,NO,N,,,1,NO,N,NO,N,,NaN,ANOREXIA,Anorexia,Decreased appetite,Appetite disorders,Appetite and general nutritional disorders,Metabolism and nutrition disorders,None,None,YES,Y,N,NO,3,2,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED
1,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,2,5.560000e+14,,,,ADVERSE EVENT,1,FEBRILE NEUTROPENIA,,None,2018-06-13,6/13/2018,,,None,2018-06-15,6/15/2018,,NO,N,,,2,NO,N,YES,Y,,NaN,FEBRILE NEUTROPENIA,Febrile neutropenia,Febrile neutropenia,Neutropenias,White blood cell disorders,Blood and lymphatic system disorders,None,None,YES,Y,N,NO,4,3,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED
2,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,3,5.560000e+14,,,,ADVERSE EVENT,1,NEUTROPHIL COUNT DECREASED,,None,2018-06-08,6/8/2018,,,None,2018-06-13,6/13/2018,,NO,N,,,3,NO,N,NO,N,,NaN,NEUTROPHIL COUNT DECREASED,Neutrophil count decreased,Neutrophil count decreased,White blood cell analyses,Haematology investigations (incl blood groups),Investigations,None,None,YES,Y,N,NO,5,4,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED
3,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,4,5.560000e+14,,,,ADVERSE EVENT,1,ECZEMA(FOOT),,None,2018-06-08,6/8/2018,,,None,2018-06-14,6/14/2018,,NO,N,,,4,NO,N,YES,Y,,NaN,ECZEMA(FOOT),Eczema,Eczema,Dermatitis and eczema,Epidermal and dermal conditions,Skin and subcutaneous tissue disorders,None,None,YES,Y,N,NO,2,1,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED
4,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,5,5.560000e+14,,,,ADVERSE EVENT,1,MALAISE,,None,2018-06-08,6/8/2018,,,None,2018-06-15,6/15/2018,,NO,N,,,5,NO,N,NO,N,,NaN,MALAISE,Malaise,Malaise,Asthenic conditions,General system disorders NEC,General disorders and administration site cond...,None,None,YES,Y,N,NO,3,2,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED
5,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,6,5.560000e+14,,,,ADVERSE EVENT,1,SORE THROAT,,None,2018-06-13,6/13/2018,,,None,2018-06-21,6/21/2018,,NO,N,,,6,NO,N,YES,Y,,NaN,SORE THROAT,Sore throat,Oropharyngeal pain,Upper respiratory tract signs and symptoms,Respiratory tract signs and symptoms,"Respiratory, thoracic and mediastinal disorders",None,None,YES,Y,N,NO,3,2,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED
6,StudyH

In [22]:
# getting the list of subject IDs that have taken concomitant medication
subjects_cm = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/list')
subjects_cm_json= subjects_cm.json()
subjects_cm = pd.DataFrame(subjects_cm_json['data'], columns = ['subject_id'])

In [23]:
subjects_cm.head()

,subject_id
0,76809
1,130301
2,132645
3,138700
4,166621
5,168918
6,205861
7,226507
8,226653
9,231407


In [10]:
# getting CM data for one subject ID (2660588)
response_cm = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/2660588/list')
response_cm_json = response_cm.json()
cm_df = pd.DataFrame(response_cm_json['data'])

In [11]:
# converting the date column into a readable format to clean
cm_df['cmstdat'] = pd.to_datetime(cm_df['cmstdat'], errors = 'coerce', format = '%d-%b-%y')
cm_df['cmendat'] = pd.to_datetime(cm_df['cmendat'], errors = 'coerce', format = '%d-%b-%y')
cm_df.head()

,studyid,siteid,subjid,subjectid,visitid,visitidx,visitnam,formname,formid,formidx,formindex,cmyn,cmyn_c,cmcat_c,cmcat_nd,cmscat,cmscat_c,cmpresp,cmpresp_c,cmpresp_nd,cmtrt,cmtrt_nd,cmrgm,cmrgm_nd,cmindc,cmaeno,cmaer,cmaer_c,cmdostot,cmdostot_nd,cmdosu,cmdosu_c,cmdosu_nd,cmroute,cmroute_c,cmprior,cmprior_c,cmstdat,cmstdat_dtr,cmstdat_dts,cmendat,cmendat_dtr,cmendat_dts,cmongo,cmongo_c,cmongo_nd,compterm,cmdecod,siteno
0,StudyHack,5086,50865017,2660588,38942,1,AECM,CM201,38623,1,5.780000e+14,YES,Y,1,,,,NO,N,,FOSAPREPITANT,,,,,,NO,N,530.0,,mg,8.0,,,None,NO,N,2019-03-22,3/22/2019,3/22/2019,2019-03-22,3/22/2019,3/22/2019,NO,N,,FOSAPREPITANT,FOSAPREPITANT,523266
1,StudyHack,5086,50865017,2660588,38942,1,AECM,CM201,38623,2,5.830000e+14,YES,Y,1,,,,NO,N,,ACYCLOVIR,,,,,,NO,N,400.0,,mg,8.0,,,None,NO,N,2019-03-22,3/22/2019,3/22/2019,2019-03-27,3/27/2019,3/27/2019,NO,N,,ACYCLOVIR,ACICLOVIR,523266
2,StudyHack,5086,50865017,2660588,38942,1,AECM,CM201,38623,3,5.830000e+14,YES,Y,1,,,,NO,N,,BIOTENE DRY MOUTH RINSE,,,,,,NO,N,5.0,,mL,9.0,,,None,NO,N,2019-03-22,3/22/2019,3/22/2019,2019-03-27,3/27/2019,3/27/2019,NO,N,,BIOTENE DRY MOUTH RINSE,GLUCOSE OXIDASE;LACTOFERRIN;LACTOPEROXIDASE;LY...,523266
3,StudyHack,5086,50865017,2660588,38942,1,AECM,CM201,38623,4,6.140000e+14,YES,Y,1,,,,NO,N,,ACETAMINOPHEN,,,,,7,YES,Y,NaN,,,NaN,,,None,NO,N,2019-04-13,4/13/2019,4/13/2019,2019-04-13,4/13/2019,4/13/2019,NO,N,,ACETAMINOPHEN,PARACETAMOL,523266


In [12]:
# getting CM data for second subject ID (130301)
response_cm2 = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/130301/list')
response_cm_json2 = response_cm2.json()
cm_df2 = pd.DataFrame(response_cm_json2['data'])

In [13]:
# converting the date column into a readable format to clean
cm_df2['cmstdat'] = pd.to_datetime(cm_df2['cmstdat'], errors = 'coerce', format = '%d-%b-%y')
cm_df2['cmendat'] = pd.to_datetime(cm_df2['cmendat'], errors = 'coerce', format = '%d-%b-%y')
cm_df2.head()

,studyid,siteid,subjid,subjectid,visitid,visitidx,visitnam,formname,formid,formidx,formindex,cmyn,cmyn_c,cmcat_c,cmcat_nd,cmscat,cmscat_c,cmpresp,cmpresp_c,cmpresp_nd,cmtrt,cmtrt_nd,cmrgm,cmrgm_nd,cmindc,cmaeno,cmaer,cmaer_c,cmdostot,cmdostot_nd,cmdosu,cmdosu_c,cmdosu_nd,cmroute,cmroute_c,cmprior,cmprior_c,cmstdat,cmstdat_dtr,cmstdat_dts,cmendat,cmendat_dtr,cmendat_dts,cmongo,cmongo_c,cmongo_nd,compterm,cmdecod,siteno
0,StudyHack,5535,55355001,130301,38942,1,AECM,CM201_3,32993,1,5.560000e+14,YES,Y,8,,,,NO,N,,HYDROXYCARBAMIDE,,1,,,,,,NaN,,,NaN,,,None,NO,N,2018-07-23,,7/23/2018,2018-07-25,,7/25/2018,NO,N,,HYDROXYCARBAMIDE,HYDROXYCARBAMIDE,53053
1,StudyHack,5535,55355001,130301,38942,1,AECM,CM201_3,32993,2,6.180000e+14,YES,Y,8,,,,NO,N,,Cytarabine,,2,,,,,,NaN,,,NaN,,,None,NO,N,2018-07-25,,7/25/2018,2018-08-07,,8/7/2018,NO,N,,Cytarabine,CYTARABINE,53053
2,StudyHack,5535,55355001,130301,38942,1,AECM,CM201_3,32993,3,6.180000e+14,YES,Y,8,,,,NO,N,,Lenograstim,,3,,,,,,NaN,,,NaN,,,None,NO,N,2018-07-25,,7/25/2018,2018-08-07,,8/7/2018,NO,N,,Lenograstim,LENOGRASTIM,53053
3,StudyHack,5535,55355001,130301,38942,1,AECM,CM201_3,32993,4,6.180000e+14,YES,Y,8,,,,NO,N,,Aclarubicin Hydrochloride,,4,,,,,,NaN,,,NaN,,,None,NO,N,2018-07-25,,7/25/2018,2018-07-28,,7/28/2018,NO,N,,Aclarubicin Hydrochloride,ACLARUBICIN HYDROCHLORIDE,53053
4,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,1,5.550000e+14,YES,Y,1,,,,NO,N,,CODEINE PHOSPHATE HYDRATE,,,,,16,YES,Y,3.0,,g,45.0,,,None,NO,N,NaT,7/50/2018,7/50/2018,2018-07-23,7/23/2018,7/23/2018,NO,N,,CODEINE PHOSPHATE HYDRATE,CODEINE PHOSPHATE,53053
5,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,2,5.550000e+14,YES,Y,1,,,,NO,N,,ESOMEPRAZOLE MAGNESIUM HYDRATE,,,,,,NO,N,50.0,,mg,8.0,,,None,NO,N,2018-06-06,6/6/2018,6/6/2018,NaT,,,YES,Y,,ESOMEPRAZOLE MAGNESIUM HYDRATE,ESOMEPRAZOLE MAGNESIUM,53053
6,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,3,5.550000e+14,YES,Y,1,,,,NO,N,,BROTIZOLAM,,,,,,NO,N,250.0,,ug,43.0,,,None,YES,Y,NaT,2003/UNK/UNK,2003,NaT,,,YES,Y,,BROTIZOLAM,BROTIZOLAM,53053
7,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,4,5.550000e+14,YES,Y,1,,,,NO,N,,ROSUVASTATIN CALCIUM,,,,,,NO,N,2.0,,mg,8.0,,,None,YES,Y,NaT,2008/UNK/UNK,2008,NaT,,,YES,Y,,ROSUVASTATIN CALCIUM,ROSUVASTATIN CALCIUM,53053
8,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,5,5.550000e+14,YES,Y,1,,,,NO,N,,CARBAZOCHROME SODIUM SULFONATE HYDRATE,,,,,"51,52",YES,Y,500.0,,mg,8.0,,,None,NO,N,2018-06-29,6/29/2018,6/29/2018,2018-07-07,7/7/2018,7/7/2018,NO,N,,CARBAZOCHROME SODIUM SULFONATE HYDRATE,CARBAZOCHROME SODIUM SULFONATE,53053
9,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,6,5.550000e+14,YES,Y,1,,,,NO,N,,POTASSIUM L-ASPARTATE,,,,,7,YES,Y,500.0,,mEq,77.0,,,None,NO,N,2018-06-26,6/26/2018,6/26/2018,2018-07-18,7/18/2018,7/18/2018,NO,N,,POTASSIUM L-ASPARTATE,POTASSIUM ASPARTATE,53053


In [14]:
# creating a dataframe consisting of the start dates of AE and CM for ID 130301
type1_df = pd.DataFrame({'aestdat':ae_df2['aestdat'],'cmstdat':cm_df2['cmstdat']})

# condition for type 1 discrepancy
type1_df['discrepancy'] = type1_df['cmstdat'] < type1_df['aestdat']

# cleaning the nat values and false values for the above condition
type1_df = type1_df.dropna(axis=0) 
type1_df.drop(type1_df[type1_df['discrepancy'] == False].index)

,aestdat,cmstdat,discrepancy
5,2018-06-13,2018-06-06,True
9,2018-06-29,2018-06-26,True
10,2018-06-29,2018-06-26,True
14,2018-06-07,2018-05-25,True
16,2018-06-14,2018-05-30,True
17,2018-06-20,2018-05-30,True
18,2018-07-11,2018-06-06,True
19,2018-07-11,2018-06-06,True
20,2018-07-11,2018-06-26,True
22,2018-07-18,2018-06-09,True


In [15]:
# type 1 discrepancy result for subject ID 130301
type1 = pd.concat([ae_df2, cm_df2], axis=1, join='inner')
type1.iloc[[5,9,10,14,16,17,18,19,20,22,23]]

,studyid,siteid,subjid,subjectid,visitindex,visitnam,formname,formid,formidx,formindex,dov,aemeyn,aemeyn_c,aecat,aecat_c,aeterm,aemedrg,aeinpkg,aestdat,aestdat_dts,aestdat_nd,aestdme,aestdme_c,aeendat,aeendat_dts,aeendat_nd,aedis,aedis_c,aemerae,aemerae_c,aespid,aendgiv,aendgiv_c,aecmgiv,aecmgiv_c,aecmgiv_nd,aerefid,compterm,aellt,aedecod,aehlt,aehlgt,aesoc,aemedrg_c,aespid_1,aeyn,aeyn_c,aeongo_c,aeongo,aetoxgr_c,aetoxgr,aeser_c,aeser,aerme_c,aerme,aetrt1,aetrt3_c,aerel1_c,aerel1,aerel1_nd,aeacn1_c,aeacn1,aeacn1_1_c,aeacn1_1,aetrt2,aetrt2_nd,aetrt2_c,aerel2_c,aerel2,aerel2_nd,aeacn2_c,aeacn2,aeacn2_1_c,aeacn2_1,aetrt3,aetrt1_c,aerel3_c,aerel3,aeacn3_c,aeacn3,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout,studyid,siteid,subjid,subjectid,visitid,visitidx,visitnam,formname,formid,formidx,formindex,cmyn,cmyn_c,cmcat_c,cmcat_nd,cmscat,cmscat_c,cmpresp,cmpresp_c,cmpresp_nd,cmtrt,cmtrt_nd,cmrgm,cmrgm_nd,cmindc,cmaeno,cmaer,cmaer_c,cmdostot,cmdostot_nd,cmdosu,cmdosu_c,cmdosu_nd,cmroute,cmroute_c,cmprior,cmprior_c,cmstdat,cmstdat_dtr,cmstdat_dts,cmendat,cmendat_dtr,cmendat_dts,cmongo,cmongo_c,cmongo_nd,compterm,cmdecod,siteno
5,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,6,5.560000e+14,,,,ADVERSE EVENT,1,SORE THROAT,,None,2018-06-13,6/13/2018,,,None,2018-06-21,6/21/2018,,NO,N,,,6,NO,N,YES,Y,,NaN,SORE THROAT,Sore throat,Oropharyngeal pain,Upper respiratory tract signs and symptoms,Respiratory tract signs and symptoms,"Respiratory, thoracic and mediastinal disorders",None,None,YES,Y,N,NO,3,2,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,2,5.550000e+14,YES,Y,1,,,,NO,N,,ESOMEPRAZOLE MAGNESIUM HYDRATE,,,,,,NO,N,50.0,,mg,8.0,,,None,NO,N,2018-06-06,6/6/2018,6/6/2018,NaT,,,YES,Y,,ESOMEPRAZOLE MAGNESIUM HYDRATE,ESOMEPRAZOLE MAGNESIUM,53053
9,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,12,5.560000e+14,,,,ADVERSE EVENT,1,RIGHT EXTERNAL EAR INFLAMMATION(RIGHT),,None,2018-06-29,6/29/2018,,,None,2018-07-23,7/23/2018,,NO,N,,,12,NO,N,YES,Y,,NaN,RIGHT EXTERNAL EAR INFLAMMATION(RIGHT),External ear inflammation,External ear inflammation,External ear infections and inflammations,External ear disorders (excl congenital),Ear and labyrinth disorders,None,None,YES,Y,N,NO,2,1,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug1,1,1,NOT RELATED,NaN,,2.0,DOSE NOT CHANGED,3.0,OTHER,,Straining during bronchoscopic examination,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,6,5.550000e+14,YES,Y,1,,,,NO,N,,POTASSIUM L-ASPARTATE,,,,,7,YES,Y,500.0,,mEq,77.0,,,None,NO,N,2018-06-26,6/26/2018,6/26/2018,2018-07-18,7/18/2018,7/18/2018,NO,N,,POTASSIUM L-ASPARTATE,POTASSIUM ASPARTATE,53053
10,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,13,5.560000e+14,,,,ADVERSE EVENT,1,HEARING LOSSES(RIGHT),,None,2018-06-29,6/29/2018,,,None,2018-07-23,7/23/2018,,NO,N,,,13,NO,N,NO,N,,NaN,HEARING LOSSES(RIGHT),Hearing loss unilateral,Deafness unilateral,Hearing losses,Hearing disorders,Ear and labyrinth disorders,None,None,YES,Y,N,NO,2,1,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug1,1,1,NOT RELATED,NaN,,2.0,DOSE NOT CHANGED,3.0,OTHER,,Straining during bronchoscopic examination,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,7,5.550000e+14,YES,Y,1,,,,NO,N,,AMPHOTERICIN B,,,,,18,YES,Y,530.0,,mg,8.0,,,None,NO,N,2018-06-26,6/26/2018,6/26/2018,2018-07-02,7/2/2018,7/2/2018,NO,N,,AMPHOTERICIN B,AMPHOTERICIN B,53053
14,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,17,5.560000e+14,,,,ADVERSE EVENT,1,PLATELET COUNT DECREASED,,None,2018-06-07,6/7/2018,,,None,NaT,,,NO,N,,,17,YES,Y,NO,N,,NaN,PLATELET COUNT DECREASED,Platelet count dec

In [16]:
# creating a dataframe consisting of the start date of CM and end date of AE for ID 130301
type2_df = pd.DataFrame({'aeendat':ae_df2['aeendat'],'cmstdat':cm_df2['cmstdat']})

# condition for type 2 discrepancy
type2_df['discrepancy'] = type2_df['cmstdat'] > type2_df['aeendat']

# cleaning nat values and false values for the above condition
type2_df = type2_df.dropna(axis=0) 
type2_df.drop(type2_df[type2_df['discrepancy'] == False].index)

,aeendat,cmstdat,discrepancy
0,2018-06-16,2018-07-23,True
1,2018-06-15,2018-07-25,True
2,2018-06-13,2018-07-25,True
3,2018-06-14,2018-07-25,True
24,2018-06-27,2018-06-30,True


In [17]:
# type 2 discrepancy result for subject ID 130301
type2 = pd.concat([ae_df2, cm_df2], axis=1, join='inner')
type2.iloc[[0,1,2,3,24]]

,studyid,siteid,subjid,subjectid,visitindex,visitnam,formname,formid,formidx,formindex,dov,aemeyn,aemeyn_c,aecat,aecat_c,aeterm,aemedrg,aeinpkg,aestdat,aestdat_dts,aestdat_nd,aestdme,aestdme_c,aeendat,aeendat_dts,aeendat_nd,aedis,aedis_c,aemerae,aemerae_c,aespid,aendgiv,aendgiv_c,aecmgiv,aecmgiv_c,aecmgiv_nd,aerefid,compterm,aellt,aedecod,aehlt,aehlgt,aesoc,aemedrg_c,aespid_1,aeyn,aeyn_c,aeongo_c,aeongo,aetoxgr_c,aetoxgr,aeser_c,aeser,aerme_c,aerme,aetrt1,aetrt3_c,aerel1_c,aerel1,aerel1_nd,aeacn1_c,aeacn1,aeacn1_1_c,aeacn1_1,aetrt2,aetrt2_nd,aetrt2_c,aerel2_c,aerel2,aerel2_nd,aeacn2_c,aeacn2,aeacn2_1_c,aeacn2_1,aetrt3,aetrt1_c,aerel3_c,aerel3,aeacn3_c,aeacn3,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout,studyid,siteid,subjid,subjectid,visitid,visitidx,visitnam,formname,formid,formidx,formindex,cmyn,cmyn_c,cmcat_c,cmcat_nd,cmscat,cmscat_c,cmpresp,cmpresp_c,cmpresp_nd,cmtrt,cmtrt_nd,cmrgm,cmrgm_nd,cmindc,cmaeno,cmaer,cmaer_c,cmdostot,cmdostot_nd,cmdosu,cmdosu_c,cmdosu_nd,cmroute,cmroute_c,cmprior,cmprior_c,cmstdat,cmstdat_dtr,cmstdat_dts,cmendat,cmendat_dtr,cmendat_dts,cmongo,cmongo_c,cmongo_nd,compterm,cmdecod,siteno
0,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,1,5.550000e+14,,,,ADVERSE EVENT,1,ANOREXIA,,None,2018-06-08,6/8/2018,,,None,2018-06-16,6/16/2018,,NO,N,,,1,NO,N,NO,N,,NaN,ANOREXIA,Anorexia,Decreased appetite,Appetite disorders,Appetite and general nutritional disorders,Metabolism and nutrition disorders,None,None,YES,Y,N,NO,3,2,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201_3,32993,1,5.560000e+14,YES,Y,8,,,,NO,N,,HYDROXYCARBAMIDE,,1,,,,,,NaN,,,NaN,,,None,NO,N,2018-07-23,,7/23/2018,2018-07-25,,7/25/2018,NO,N,,HYDROXYCARBAMIDE,HYDROXYCARBAMIDE,53053
1,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,2,5.560000e+14,,,,ADVERSE EVENT,1,FEBRILE NEUTROPENIA,,None,2018-06-13,6/13/2018,,,None,2018-06-15,6/15/2018,,NO,N,,,2,NO,N,YES,Y,,NaN,FEBRILE NEUTROPENIA,Febrile neutropenia,Febrile neutropenia,Neutropenias,White blood cell disorders,Blood and lymphatic system disorders,None,None,YES,Y,N,NO,4,3,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201_3,32993,2,6.180000e+14,YES,Y,8,,,,NO,N,,Cytarabine,,2,,,,,,NaN,,,NaN,,,None,NO,N,2018-07-25,,7/25/2018,2018-08-07,,8/7/2018,NO,N,,Cytarabine,CYTARABINE,53053
2,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,3,5.560000e+14,,,,ADVERSE EVENT,1,NEUTROPHIL COUNT DECREASED,,None,2018-06-08,6/8/2018,,,None,2018-06-13,6/13/2018,,NO,N,,,3,NO,N,NO,N,,NaN,NEUTROPHIL COUNT DECREASED,Neutrophil count decreased,Neutrophil count decreased,White blood cell analyses,Haematology investigations (incl blood groups),Investigations,None,None,YES,Y,N,NO,5,4,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201_3,32993,3,6.180000e+14,YES,Y,8,,,,NO,N,,Lenograstim,,3,,,,,,NaN,,,NaN,,,None,NO,N,2018-07-25,,7/25/2018,2018-08-07,,8/7/2018,NO,N,,Lenograstim,LENOGRASTIM,53053
3,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,4,5.560000e+14,,,,ADVERSE EVENT,1,ECZEMA(FOOT),,None,2018-06-08,6/8/2018,,,None,2018-06-14,6/14/2018,,NO,N,,,4,NO,N,YES,Y,,NaN,ECZEMA(FOOT),Eczema,Eczema,Dermatitis and eczema,Epidermal and dermal conditions,Skin and subcutaneous tissue disorders,None,None,YES,Y,N,NO,2,1,N,NO,N,NO,BLINDED THERAPY,3,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,1

In [18]:
# type 3 discrepancy result for subject ID 2660588
type3 = ae_df[ae_df.duplicated(['aestdat', 'aeterm'])]
type3

,studyid,siteid,subjid,subjectid,visitindex,visitnam,formname,formid,formidx,formindex,dov,aemeyn,aemeyn_c,aecat,aecat_c,aeterm,aemedrg,aeinpkg,aestdat,aestdat_dts,aestdat_nd,aestdme,aestdme_c,aeendat,aeendat_dts,aeendat_nd,aedis,aedis_c,aemerae,aemerae_c,aespid,aendgiv,aendgiv_c,aecmgiv,aecmgiv_c,aecmgiv_nd,aerefid,compterm,aellt,aedecod,aehlt,aehlgt,aesoc,aemedrg_c,aespid_1,aeyn,aeyn_c,aeongo_c,aeongo,aetoxgr_c,aetoxgr,aeser_c,aeser,aerme_c,aerme,aetrt1,aetrt3_c,aerel1_c,aerel1,aerel1_nd,aeacn1_c,aeacn1,aeacn1_1_c,aeacn1_1,aetrt2,aetrt2_nd,aetrt2_c,aerel2_c,aerel2,aerel2_nd,aeacn2_c,aeacn2,aeacn2_1_c,aeacn2_1,aetrt3,aetrt1_c,aerel3_c,aerel3,aeacn3_c,aeacn3,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout
23,StudyHack,5086,50865017,2660588,1,AECM,AE202,38768,21,6.150000e+14,,,,ADVERSE EVENT,1,ANEMIA,,None,2019-04-14,4/14/2019,,,None,2019-04-17,4/17/2019,,NO,N,,,21,NO,N,NO,N,,NaN,ANEMIA,Anemia,Anaemia,Anaemias NEC,Anaemias nonhaemolytic and marrow depression,Blood and lymphatic system disorders,None,None,YES,Y,N,NO,3.0,2.0,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,2,RELATED,,2.0,DOSE NOT CHANGED,NaN,,Studydrug1,1,2,RELATED,2.0,DOSE NOT CHANGED,NaN,,NaN,,,,5.0,RECOVERING/RESOLVING


In [19]:
# type 4 discrepancy result for subject ID 130301
type4 = cm_df2[cm_df2.duplicated(['cmtrt', 'cmstdat'])]
type4

,studyid,siteid,subjid,subjectid,visitid,visitidx,visitnam,formname,formid,formidx,formindex,cmyn,cmyn_c,cmcat_c,cmcat_nd,cmscat,cmscat_c,cmpresp,cmpresp_c,cmpresp_nd,cmtrt,cmtrt_nd,cmrgm,cmrgm_nd,cmindc,cmaeno,cmaer,cmaer_c,cmdostot,cmdostot_nd,cmdosu,cmdosu_c,cmdosu_nd,cmroute,cmroute_c,cmprior,cmprior_c,cmstdat,cmstdat_dtr,cmstdat_dts,cmendat,cmendat_dtr,cmendat_dts,cmongo,cmongo_c,cmongo_nd,compterm,cmdecod,siteno
36,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,43,5.560000e+14,YES,Y,1,,,,NO,N,,TEICOPLANIN,,,,,,NO,N,600.0,,mg,8.0,,,None,NO,N,NaT,6/53/2018,6/53/2018,2018-06-29,6/29/2018,6/29/2018,NO,N,,TEICOPLANIN,TEICOPLANIN,53053
49,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,73,5.590000e+14,YES,Y,1,,,,NO,N,,ACETAMINOPHEN,,,,,,NO,N,200.0,,mg,8.0,,,None,NO,N,NaT,6/53/2018,6/53/2018,NaT,,,YES,Y,,ACETAMINOPHEN,PARACETAMOL,53053
57,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,81,5.730000e+14,YES,Y,1,,,,NO,N,,MICAFUNGIN SODIUM,,,,,,NO,N,530.0,,mg,8.0,,,None,NO,N,NaT,6/53/2018,6/53/2018,2018-06-26,6/26/2018,6/26/2018,NO,N,,MICAFUNGIN SODIUM,MICAFUNGIN SODIUM,53053


In [20]:
# creating a dataframe consisting of the end dates of AE and CM for ID 130301
type5_df = pd.DataFrame({'aeendat':ae_df2['aeendat'], 'cmendat':cm_df2['cmendat']})

# condition for type 5 discrepancy
type5_df['discrepancy'] = type5_df['aeendat'] > type5_df['cmendat']

# cleaning nat values and false values for the above condition
type5_df = type5_df.dropna(axis=0) 
type5_df.drop(type5_df[type5_df['discrepancy'] == False].index)

,aeendat,cmendat,discrepancy
9,2018-07-23,2018-07-18,True
10,2018-07-23,2018-07-02,True
11,2018-07-17,2018-07-13,True
12,2018-07-19,2018-06-13,True
16,2018-06-20,2018-06-13,True
17,2018-07-11,2018-06-05,True
20,2018-07-13,2018-06-26,True
21,2018-07-18,2018-06-14,True


In [21]:
# type 5 discrepancy result for subject ID 130301
type5_df = pd.concat([ae_df2, cm_df2], axis=1, join='inner')
type5_df.iloc[[9,10,11,12,16,17,20,21]]

,studyid,siteid,subjid,subjectid,visitindex,visitnam,formname,formid,formidx,formindex,dov,aemeyn,aemeyn_c,aecat,aecat_c,aeterm,aemedrg,aeinpkg,aestdat,aestdat_dts,aestdat_nd,aestdme,aestdme_c,aeendat,aeendat_dts,aeendat_nd,aedis,aedis_c,aemerae,aemerae_c,aespid,aendgiv,aendgiv_c,aecmgiv,aecmgiv_c,aecmgiv_nd,aerefid,compterm,aellt,aedecod,aehlt,aehlgt,aesoc,aemedrg_c,aespid_1,aeyn,aeyn_c,aeongo_c,aeongo,aetoxgr_c,aetoxgr,aeser_c,aeser,aerme_c,aerme,aetrt1,aetrt3_c,aerel1_c,aerel1,aerel1_nd,aeacn1_c,aeacn1,aeacn1_1_c,aeacn1_1,aetrt2,aetrt2_nd,aetrt2_c,aerel2_c,aerel2,aerel2_nd,aeacn2_c,aeacn2,aeacn2_1_c,aeacn2_1,aetrt3,aetrt1_c,aerel3_c,aerel3,aeacn3_c,aeacn3,aeacn3_1_c,aeacn3_1,aerelnst_c,aerelnst,aerelnst_nd,aerelt,aeout_c,aeout,studyid,siteid,subjid,subjectid,visitid,visitidx,visitnam,formname,formid,formidx,formindex,cmyn,cmyn_c,cmcat_c,cmcat_nd,cmscat,cmscat_c,cmpresp,cmpresp_c,cmpresp_nd,cmtrt,cmtrt_nd,cmrgm,cmrgm_nd,cmindc,cmaeno,cmaer,cmaer_c,cmdostot,cmdostot_nd,cmdosu,cmdosu_c,cmdosu_nd,cmroute,cmroute_c,cmprior,cmprior_c,cmstdat,cmstdat_dtr,cmstdat_dts,cmendat,cmendat_dtr,cmendat_dts,cmongo,cmongo_c,cmongo_nd,compterm,cmdecod,siteno
9,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,12,5.560000e+14,,,,ADVERSE EVENT,1,RIGHT EXTERNAL EAR INFLAMMATION(RIGHT),,None,2018-06-29,6/29/2018,,,None,2018-07-23,7/23/2018,,NO,N,,,12,NO,N,YES,Y,,NaN,RIGHT EXTERNAL EAR INFLAMMATION(RIGHT),External ear inflammation,External ear inflammation,External ear infections and inflammations,External ear disorders (excl congenital),Ear and labyrinth disorders,None,None,YES,Y,N,NO,2,1,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug1,1,1,NOT RELATED,NaN,,2.0,DOSE NOT CHANGED,3.0,OTHER,,Straining during bronchoscopic examination,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,6,5.550000e+14,YES,Y,1,,,,NO,N,,POTASSIUM L-ASPARTATE,,,,,7,YES,Y,500.0,,mEq,77.0,,,None,NO,N,2018-06-26,6/26/2018,6/26/2018,2018-07-18,7/18/2018,7/18/2018,NO,N,,POTASSIUM L-ASPARTATE,POTASSIUM ASPARTATE,53053
10,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,13,5.560000e+14,,,,ADVERSE EVENT,1,HEARING LOSSES(RIGHT),,None,2018-06-29,6/29/2018,,,None,2018-07-23,7/23/2018,,NO,N,,,13,NO,N,NO,N,,NaN,HEARING LOSSES(RIGHT),Hearing loss unilateral,Deafness unilateral,Hearing losses,Hearing disorders,Ear and labyrinth disorders,None,None,YES,Y,N,NO,2,1,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug3,,2,1,NOT RELATED,,NaN,,2.0,DOSE NOT CHANGED,Studydrug1,1,1,NOT RELATED,NaN,,2.0,DOSE NOT CHANGED,3.0,OTHER,,Straining during bronchoscopic examination,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,7,5.550000e+14,YES,Y,1,,,,NO,N,,AMPHOTERICIN B,,,,,18,YES,Y,530.0,,mg,8.0,,,None,NO,N,2018-06-26,6/26/2018,6/26/2018,2018-07-02,7/2/2018,7/2/2018,NO,N,,AMPHOTERICIN B,AMPHOTERICIN B,53053
11,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,14,5.560000e+14,,,,ADVERSE EVENT,1,WHEALS,,None,2018-07-07,7/7/2018,,,None,2018-07-17,7/17/2018,,NO,N,,,14,NO,N,YES,Y,,NaN,WHEALS,Wheals,Urticaria,Urticarias,Angioedema and urticaria,Skin and subcutaneous tissue disorders,None,None,YES,Y,N,NO,2,1,N,NO,N,NO,BLINDED THERAPY,3,1,NOT RELATED,,NaN,,6.0,NOT APPLICABLE,Studydrug3,,2,1,NOT RELATED,,NaN,,6.0,NOT APPLICABLE,Studydrug1,1,1,NOT RELATED,NaN,,6.0,NOT APPLICABLE,1.0,CONCOMITANT DRUG TREATMENT,,,3,RECOVERED/RESOLVED,StudyHack,5535,55355001,130301,38942,1,AECM,CM201,38623,8,5.550000e+14,YES,Y,1,,,,NO,N,,ZOSYN,,,,,16,YES,Y,13.0,,g,45.0,,,None,NO,N,2018-07-08,7/8/2018,7/8/2018,2018-07-13,7/13/2018,7/13/2018,NO,N,,ZOSYN,PIPERACILLIN SODIUM;TAZOBACTAM SODIUM,53053
12,StudyHack,5535,55355001,130301,1,AECM,AE202,38768,15,5.560000e+14,,,,ADVERSE EVENT,1,LEFT BURSITISOLECRANON,,None,2018-07-08,7/8/2018,,,None,2018-07-19,7/19/2018,,NO,N,,,15,NO,N,NO,N,,NaN,LEFT BURSITISOLECRANON,Olecranon bursitis,Bursitis,Bursal disorders,Synovial and bu